In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import os
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset,DataLoader

In [2]:
#check gpu device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [3]:
# load data
dir = 'E:\\Sebnewrepo/Rec_sys_lab/paper1_experiment/'
checkin_file = 'ny_ordered.csv'
df = pd.read_csv(dir + checkin_file)
df.head()

user_id                    poi_id           poi_category_id  \
0        1  4abc1f51f964a520798620e3  4bf58dd8d48988d1ce941735   
1        1  4d4ac10da0ef54814b6ffff6  4bf58dd8d48988d157941735   
2        1  4db44994cda1c57c82583709  4bf58dd8d48988d1f1931735   
3        1  4a541923f964a52008b31fe3  4bf58dd8d48988d14e941735   
4        1  40f1d480f964a5205b0a1fe3  4bf58dd8d48988d143941735   

       poi_category_name   latitude  longitude  time_offset  \
0     Seafood Restaurant  40.781558 -73.975792         -240   
1    American Restaurant  40.784018 -73.974524         -240   
2  General Entertainment  40.739398 -73.993210         -240   
3    American Restaurant  40.785677 -73.976498         -240   
4         Breakfast Spot  40.719929 -74.008532         -240   

                         UTC_time             datetime  
0  Wed Apr 04 23:31:31 +0000 2012  2012-04-04 23:31:31  
1  Sat Apr 07 17:42:24 +0000 2012  2012-04-07 17:42:24  
2  Sun Apr 08 18:20:29 +0000 2012  2012-04-08 18:20:29  
3  Sun Apr 08 20:02:10 +0000 2012  2012-04-08 20:02:10  
4  Mon Apr 09 16:20:52 +0000 2012  2012-04-09 16:20:52

In [4]:
# POIs encode, and generate encode mapping
poi_cat = pd.Categorical(df['poi_id'])
poi_encode = poi_cat.codes
#generate poi mapping table
poi_mapping = pd.DataFrame({
    'poi_encode': poi_encode,
    'poi_id': df['poi_id']
    })
#drop duplicate
poi_mapping_output = poi_mapping.drop_duplicates()
df['poi_encode'] = poi_encode
df.drop(['poi_id'], axis = 1, inplace = True)
df.head(5)

user_id           poi_category_id      poi_category_name   latitude  \
0        1  4bf58dd8d48988d1ce941735     Seafood Restaurant  40.781558   
1        1  4bf58dd8d48988d157941735    American Restaurant  40.784018   
2        1  4bf58dd8d48988d1f1931735  General Entertainment  40.739398   
3        1  4bf58dd8d48988d14e941735    American Restaurant  40.785677   
4        1  4bf58dd8d48988d143941735         Breakfast Spot  40.719929   

   longitude  time_offset                        UTC_time  \
0 -73.975792         -240  Wed Apr 04 23:31:31 +0000 2012   
1 -73.974524         -240  Sat Apr 07 17:42:24 +0000 2012   
2 -73.993210         -240  Sun Apr 08 18:20:29 +0000 2012   
3 -73.976498         -240  Sun Apr 08 20:02:10 +0000 2012   
4 -74.008532         -240  Mon Apr 09 16:20:52 +0000 2012   

              datetime  poi_encode  
0  2012-04-04 23:31:31        2752  
1  2012-04-07 17:42:24        7093  
2  2012-04-08 18:20:29        7455  
3  2012-04-08 20:02:10        1958  
4  2012-04-09 16:20:52         499

In [5]:
# only keep user id and sequential POIs

df_input = pd.DataFrame({
    'user_id': df['user_id'],  # user_id offset by 1
    'poi_id': df['poi_encode'],
    #'implicit': np.ones(179468)
})

In [6]:
df_input = df_input.reset_index(drop = True)
df_input

user_id  poi_id
0             1    2752
1             1    7093
2             1    7455
3             1    1958
4             1     499
...         ...     ...
179463     1083    1541
179464     1083     466
179465     1083    8703
179466     1083    9987
179467     1083    1956

[179468 rows x 2 columns]

In [7]:
df_input['user_id'].nunique()

1083

In [8]:
df_input['poi_id'].nunique()

9989

In [9]:
from copy import deepcopy 
def convert_data(data):
    df = deepcopy(data)
    data = df.groupby('user_id')['poi_id'].apply(list)
    unique_data = df.groupby('user_id')['poi_id'].nunique()
    print(data[:10])
    print(len(data))
    return data

In [10]:
seq_data = convert_data(df_input)

user_id
1     [2752, 7093, 7455, 1958, 499, 270, 9066, 1344,...
2     [8834, 1303, 1714, 7089, 4713, 1830, 4512, 192...
3     [8246, 1361, 478, 9312, 816, 1028, 6685, 1194,...
4     [3662, 3662, 3662, 3662, 3662, 8135, 3662, 298...
5     [2000, 2000, 5223, 9244, 2328, 9244, 5223, 924...
6     [8584, 9680, 8584, 6217, 2943, 6362, 8584, 376...
7     [2854, 6659, 6659, 8077, 2768, 2096, 2411, 721...
8     [2946, 2946, 2946, 2179, 2946, 2946, 2946, 294...
9     [6942, 3547, 672, 1410, 6942, 2316, 6250, 860,...
10    [7183, 532, 532, 1893, 928, 1233, 6883, 5607, ...
Name: poi_id, dtype: object
1083


### deal with the POI candidate

In [175]:
candidate = pd.read_csv('E:/Sebnewrepo/Data/paper1_data/ny_candidate.csv')

In [176]:
candidate = candidate.merge(poi_mapping_output, on = 'poi_id', how = 'left')
candidate.drop(['poi_id'], axis = 1, inplace = True)
candidate.head()

Unnamed: 0  user_id    rank  distance  poi_encode
0           0        1  7388.0    12.072          33
1           1        1  7580.0    11.800          48
2           2        1  4849.0    14.673          58
3           3        1  5851.0    11.767          64
4           4        1  6475.0    11.059          72

In [177]:
# filter by the distance
candidate = candidate[candidate['distance'] < 15]
candidate.drop(['Unnamed: 0'], axis = 1, inplace = True)
candidate.shape

(6951378, 4)

### Select top % candidate
Top % candidate is the parameters here

In [178]:
candidate['filtered_rank'] = candidate.groupby('user_id')['rank'].rank()
max_rank = candidate.groupby('user_id')['filtered_rank'].max()
max_rank = max_rank.reset_index()
candidate = candidate.merge(max_rank, on = 'user_id', how = 'left')

In [179]:
# filter the top 95% candidates
top_ratio = 0.95
candidate = candidate[candidate["filtered_rank_x"] > candidate["filtered_rank_y"]* top_ratio]
candidate.shape

(348068, 6)

In [180]:
candidate_reduction = pd.DataFrame({
    'user_id': candidate['user_id'],  # user_id offset by 1
    'poi_id': candidate['poi_encode'],
    #'implicit': np.ones(179468)
})
candidate_reduction.head()

user_id  poi_id
582         1    5700
595         1    5952
765         1    8007
769         1    8054
6328        8    5952

In [181]:
candidate_seq = convert_data(candidate_reduction)
candidate_seq = candidate_seq.tolist()

user_id
1     [5700, 5952, 8007, 8054, 7590, 4617, 9122, 111...
2     [3532, 4327, 5807, 7010, 9661, 6177, 9273, 770...
3     [1116, 1941, 3338, 4336, 4983, 7596, 4084, 505...
4     [137, 437, 1446, 1604, 2479, 2891, 3282, 3800,...
5     [6011, 535, 2730, 2965, 3318, 3410, 3532, 3786...
6     [4084, 5055, 6125, 9814, 4140, 5741, 6948, 932...
7     [981, 3575, 4657, 7043, 7414, 7648, 9122, 1116...
8     [5952, 735, 3348, 3529, 4051, 4617, 4819, 5784...
9     [8007, 7043, 1116, 1941, 3338, 4336, 4983, 759...
10    [1116, 1941, 3338, 4336, 4983, 7596, 7994, 903...
Name: poi_id, dtype: object
1083


### model and evaluation

In [182]:
class selfAtt(nn.Module):
    def __init__(self, num_user, num_item, model_args, device):
        
        super(selfAtt, self).__init__()
        
        self.args = model_args
        # init args
        self.embedding_dim = self.args.d
        embedding_dim = self.embedding_dim
        self.L = self.args.L  # sequence length
        L = self.L
        self.w = 0.15  # learnable para
        
        # define embedding
        
        self.user_embed = nn.Embedding(num_user, embedding_dim).to(device)
        self.item_embed = nn.Embedding(num_item, embedding_dim).to(device)
        self.item_embed_short = nn.Embedding(num_item, embedding_dim).to(device)
        self.item_embed_long = nn.Embedding(num_item, embedding_dim).to(device)
        self.linear1 = nn.Linear(embedding_dim, embedding_dim).to(device)
        #print(self.linear1)
        self.item_position_embed = nn.Embedding.from_pretrained(self.position_embed(L),freeze=True)
        
        # initialize
        self.user_embed.weight.data.normal_(0, 1.0/self.user_embed.embedding_dim)
        self.item_embed.weight.data.normal_(0, 1.0/self.item_embed.embedding_dim)
        self.linear1.weight.data.normal_(mean=0, std=np.sqrt(2.0 / embedding_dim))
    
    def position_embed(self, L):
        position_embedding = np.array([[pos/np.power(1000, 2.*i)/ self.embedding_dim for i in range(self.embedding_dim)]
                                      for pos in range(L)])
        position_embedding[:,0::2] = np.sin(position_embedding[:,0::2])
        position_embedding[:,1::2] = np.cos(position_embedding[:,1::2])
        t = torch.from_numpy(position_embedding).to(device)
        return t
    
    def forward(self, seq_item, user_id, target, for_pred = False):
        
        '''
        user_id
        seq_item = L item id that user interact before
        target: item target
        '''
        
        # sequential item embedding
        # seq_item = seq_item.to(device)
        item_embedding = self.item_embed(seq_item).to(device)
        # item position embedding
        position_idx = torch.range(0, self.L - 1).unsqueeze(0).expand(seq_item.size(0), -1).long().to(device)
        position_embedding = self.item_position_embed(position_idx)
        item_embedding_cat = item_embedding.float() + position_embedding.float()
        
        # self-attention network
        Q = F.relu(self.linear1(item_embedding_cat))
        K = F.relu(self.linear1(item_embedding_cat))
        #print(K.shape)
        d = torch.FloatTensor([self.embedding_dim]).to(device)
        affinity = torch.matmul(Q, torch.transpose(K, 1, 2))/torch.sqrt(d)
        
        # mask the diagonal value
        mask = torch.eye(item_embedding_cat.size(1), item_embedding_cat.size(1)).byte().to(device)
        affinity = affinity.masked_fill(mask, 0)
        #S = F.softmax(affinity)
        S = torch.sigmoid(affinity)
        #print('s', S.shape)
        attention = torch.mean(torch.matmul(S, item_embedding_cat), dim=1)
        
        # user embedding
        user_id = user_id.to(device)
        user_embedding = self.user_embed(user_id).squeeze()
        #print('user_embed', user_embedding.shape)
        # target embedding short and long note: those two embedding is different 
        
        if target is None:
            target = torch.range(0,self.num_item-1).long().unsqueeze(0).cuda()
            target_embedding_short = self.item_embed_short(target).squeeze()
            target_embedding_long = self.item_embed_long(target).squeeze()
        else:
            target_embedding_short = self.item_embed_short(target).squeeze()
            target_embedding_long = self.item_embed_long(target).squeeze()
        
        # pred
        if for_pred == False:
            user_embedding = user_embedding.unsqueeze(1).expand(-1,target.size(1),-1)
            #print('to train', user_embedding.shape)
            #print('attention before', attention.shape)
            attention = attention.unsqueeze(1).expand(-1,target.size(1),-1)
            #print('attention', attention.shape)
            y_pred = self.w* torch.sqrt(torch.sum((user_embedding - target_embedding_long)**2, dim=2)) + (1-self.w)*torch.sqrt(torch.sum((attention-target_embedding_short)**2, dim=2))
            #print('to train', y_pred)
            return y_pred
        else:
            target = target.unsqueeze(0)
            #print('target_size', target.shape)
            #print('user_embedding', user_embedding.shape)
            user_embedding = user_embedding.unsqueeze(0).expand(target.size(1), -1)
            #print('user_embedding_after', user_embedding.shape)
            #print('attention_before', attention.shape)
            attention = attention.expand(target.size(1), -1)
            #print('attention', attention.shape)
            y_pred = self.w* torch.sqrt(torch.sum((user_embedding - target_embedding_long)**2, dim=1)) + (1-self.w)*torch.sqrt(torch.sum((attention-target_embedding_short)**2, dim=1))
            return y_pred

In [183]:
from interactions import Interactions
from eval_metrics import *

import argparse
import logging
from time import time
import datetime

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)


In [184]:
def evaluation(model, train, test_set, candidate, topk=20):
    model.eval()
    num_users = train.num_users
    num_items = train.num_items
    batch_size = 500
    num_batches = int(num_users / batch_size) + 1
    user_indexes = np.arange(num_users)
    item_indexes = np.arange(num_items)
    pred_list = None
    train_matrix = train.tocsr()
    test_sequences = train.test_sequences.sequences

    for batchID in range(num_batches):
        start = batchID * batch_size
        end = start + batch_size

        if batchID == num_batches - 1:
            if start < num_users:
                end = num_users
            else:
                break

        batch_user_index = user_indexes[start:end]
        batch_test_sequences = test_sequences[batch_user_index]
        batch_test_sequences = np.atleast_2d(batch_test_sequences)
        batch_test_sequences = torch.from_numpy(batch_test_sequences).type(torch.LongTensor).to(device)
        batch_candidate = candidate[start:end]
        item_ids = torch.from_numpy(item_indexes).type(torch.LongTensor).to(device)
        #print('item_ids', item_ids)
        batch_user_ids = torch.from_numpy(np.array(batch_user_index)).type(torch.LongTensor).to(device) 
        #print('batch_user_ids', batch_user_ids)
        # get te prediction rating: for_pred = True
        rating_pred = np.empty([batch_user_ids.size(0),item_ids.size(0)])
        #print(rating_pred[.shape])
        for idx in range(batch_user_ids.size(0)):
            uid = torch.tensor([idx])
            reduct = batch_candidate[idx]
            uid_pred = model(batch_test_sequences[idx].unsqueeze(0), uid, item_ids, True)
            pred = uid_pred.cpu().detach().numpy()
            for i in range(len(reduct)):
                pred[i] = -100
            rating_pred[idx] = pred
            #print(rating_pred)
        #rating_pred = model(batch_test_sequences, batch_user_ids, item_ids, True)
        #rating_pred = rating_pred.data.numpy().copy()
        rating_pred[train_matrix[batch_user_index].toarray() > 0] = 0
        #print(rating_pred[0])
        
        # reference: https://stackoverflow.com/a/23734295, https://stackoverflow.com/a/20104162
        ind = np.argpartition(rating_pred, -topk)
        #print(ind)
        ind = ind[:, -topk:]
        arr_ind = rating_pred[np.arange(len(rating_pred))[:, None], ind]
        arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]
        batch_pred_list = ind[np.arange(len(rating_pred))[:, None], arr_ind_argsort]
        #print('batch_pred_list', batch_pred_list.shape)

        if batchID == 0:
            pred_list = batch_pred_list
        else:
            pred_list = np.append(pred_list, batch_pred_list, axis=0)
    #print(pred_list.shape)
    precision, recall, MAP, ndcg = [], [], [], []
    for k in [5, 10, 15, 20]:
        precision.append(precision_at_k(test_set, pred_list, k))
        recall.append(recall_at_k(test_set, pred_list, k))
        MAP.append(mapk(test_set, pred_list, k))
        ndcg.append(ndcg_k(test_set, pred_list, k))

    return precision, recall, MAP, ndcg

In [185]:
def negsamp_vectorized_bsearch_preverif(pos_inds, n_items, n_samp=32):
    """ Pre-verified with binary search
    `pos_inds` is assumed to be ordered
    reference: https://tech.hbc.com/2018-03-23-negative-sampling-in-numpy.html
    """
    raw_samp = np.random.randint(0, n_items - len(pos_inds), size=n_samp)
    pos_inds_adj = pos_inds - np.arange(len(pos_inds))
    neg_inds = raw_samp + np.searchsorted(pos_inds_adj, raw_samp, side='right')
    return neg_inds


def generate_negative_samples(train_matrix, num_neg=3, num_sets=10):
    neg_samples = []
    for user_id, row in enumerate(train_matrix):
        pos_ind = row.indices
        neg_sample = negsamp_vectorized_bsearch_preverif(pos_ind, train_matrix.shape[1], num_neg * num_sets)
        neg_samples.append(neg_sample)

    return np.asarray(neg_samples).reshape(num_sets, train_matrix.shape[0], num_neg)


def train_model(model, train_data, test_data, candidate, config):
    num_users = train_data.num_users
    num_items = train_data.num_items

    # convert to sequences, targets and users
    sequences_np = train_data.sequences.sequences
    #print(sequences_np)
    targets_np = train_data.sequences.targets
    users_np = train_data.sequences.user_ids
    train_matrix = train_data.tocsr()

    n_train = sequences_np.shape[0]
    logger.info("Total training records:{}".format(n_train))

    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.l2)

    record_indexes = np.arange(n_train)
    batch_size = config.batch_size
    num_batches = int(n_train / batch_size) + 1
    for epoch_num in range(config.n_iter):

        t1 = time()

        # set model to training mode
        model.train()

        np.random.shuffle(record_indexes)

        t_neg_start = time()
        negatives_np_multi = generate_negative_samples(train_matrix, config.neg_samples, config.sets_of_neg_samples)
        logger.info("Negative sampling time: {}s".format(time() - t_neg_start))

        epoch_loss = 0.0
        for batchID in range(num_batches):
            start = batchID * batch_size
            end = start + batch_size

            if batchID == num_batches - 1:
                if start < n_train:
                    end = n_train
                else:
                    break

            batch_record_index = record_indexes[start:end]

            batch_users = users_np[batch_record_index]
            batch_sequences = sequences_np[batch_record_index]
            batch_targets = targets_np[batch_record_index]
            negatives_np = negatives_np_multi[batchID % config.sets_of_neg_samples]
            batch_neg = negatives_np[batch_users]

            batch_users = torch.from_numpy(batch_users).type(torch.LongTensor).to(device)
            batch_sequences = torch.from_numpy(batch_sequences).type(torch.LongTensor).to(device)
            batch_targets = torch.from_numpy(batch_targets).type(torch.LongTensor).to(device)
            batch_negatives = torch.from_numpy(batch_neg).type(torch.LongTensor).to(device)

            items_to_predict = torch.cat((batch_targets, batch_negatives), 1)
            prediction_score = model(batch_sequences, batch_users, items_to_predict, False)
            #print(prediction_score.shape)
            (targets_prediction, negatives_prediction) = torch.split(
                prediction_score, [batch_targets.size(1), batch_negatives.size(1)], dim=1)

            # compute the hinge loss
            #loss = torch.mean(F.relu(targets_prediction - negatives_prediction + 0.5), dim=1).unsqueeze(1)
            loss = -torch.log(torch.sigmoid(targets_prediction - negatives_prediction) + 1e-8)
            loss = torch.mean(torch.sum(loss))

            epoch_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # clean the grad, 
            #optimizer.zero_grad()
        epoch_loss /= num_batches

        t2 = time()

        output_str = "Epoch %d [%.1f s]  loss=%.4f" % (epoch_num + 1, t2 - t1, epoch_loss)
        logger.info(output_str)

        
        if (epoch_num + 1) % 20 == 0:
            precision, recall, MAP, ndcg = evaluation(model, train_data, test_data, candidate, topk=20)
            logger.info(', '.join(str(e) for e in precision))
            logger.info(', '.join(str(e) for e in recall))
            logger.info(', '.join(str(e) for e in MAP))
            logger.info(', '.join(str(e) for e in ndcg))
            logger.info("Evaluation time:{}".format(time() - t2))
    logger.info("\n")
    logger.info("\n")
    #torch.save(model.state_dict(), 'car.pkl')
    #print('model arg save complete')

In [186]:
# split train test data        
def split_data_sequentially(user_records, test_radio=0.2):
    train_set = []
    test_set = []

    for item_list in user_records:
        len_list = len(item_list)
        num_test_samples = int(math.ceil(len_list * test_radio))
        train_sample = []
        test_sample = []
        for i in range(len_list - num_test_samples, len_list):
            test_sample.append(item_list[i])
            
        for place in item_list:
            if place not in set(test_sample):
                train_sample.append(place)
                
        train_set.append(train_sample)
        test_set.append(test_sample)

    return train_set, test_set
    

def generate_dataset(seq_data):
    user_records = seq_data.tolist()
    # split dataset
    train_val_set, test_set = split_data_sequentially(user_records, test_radio=0.2)
    train_set, val_set = split_data_sequentially(train_val_set, test_radio=0.1)

    return train_set, val_set, train_val_set, test_set, 1083, 9989

In [187]:
train_set, val_set, train_val_set, test_set, num_users, num_items = generate_dataset(seq_data)

In [188]:
parser = argparse.ArgumentParser()

# data arguments
parser.add_argument('--L', type=int, default=5)
parser.add_argument('--T', type=int, default=3)
# train arguments
parser.add_argument('--n_iter', type=int, default=200)
parser.add_argument('--seed', type=int, default=1234)
parser.add_argument('--batch_size', type=int, default=4096)
parser.add_argument('--learning_rate', type=float, default=1e-3)
parser.add_argument('--l2', type=float, default=1e-3)
parser.add_argument('--neg_samples', type=int, default=3)
parser.add_argument('--sets_of_neg_samples', type=int, default=50)

# model dependent arguments
parser.add_argument('--d', type=int, default=100)
config = parser.parse_args(
    args = [
        '--L', '5',
        '--T', '3',
        '--n_iter', '200',
        '--seed', '1200',
        '--batch_size', '500',
        '--learning_rate', '0.001',
        '--l2', '0.001',
        '--neg_samples', '3',
        '--sets_of_neg_samples', '30'
    ])

In [189]:
train = Interactions(train_val_set, num_users, num_items)
train.to_sequence(config.L, config.T)

logger.info(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
logger.info(config)
model = selfAtt(num_users, num_items, config, device).to(device)
train_model(model, train, test_set,candidate_seq, config)

INFO:__main__:2021-06-16 02:21:49
INFO:__main__:Namespace(L=5, T=3, batch_size=500, d=100, l2=0.001, learning_rate=0.001, n_iter=200, neg_samples=3, seed=1200, sets_of_neg_samples=30)
INFO:__main__:Total training records:54872
INFO:__main__:Negative sampling time: 0.05485391616821289s
INFO:__main__:Epoch 1 [0.7 s]  loss=1118.6299
INFO:__main__:Negative sampling time: 0.054853200912475586s
INFO:__main__:Epoch 2 [0.7 s]  loss=926.2329
INFO:__main__:Negative sampling time: 0.05584979057312012s
INFO:__main__:Epoch 3 [0.7 s]  loss=807.1997
INFO:__main__:Negative sampling time: 0.0558781623840332s
INFO:__main__:Epoch 4 [0.7 s]  loss=721.6040
INFO:__main__:Negative sampling time: 0.05436372756958008s
INFO:__main__:Epoch 5 [0.7 s]  loss=664.0343
INFO:__main__:Negative sampling time: 0.054853200912475586s
INFO:__main__:Epoch 6 [0.7 s]  loss=622.1079
INFO:__main__:Negative sampling time: 0.054856061935424805s
INFO:__main__:Epoch 7 [0.7 s]  loss=591.9751
INFO:__main__:Negative sampling time: 0.05

INFO:__main__:Epoch 64 [0.7 s]  loss=226.9524
INFO:__main__:Negative sampling time: 0.05585050582885742s
INFO:__main__:Epoch 65 [0.7 s]  loss=223.0457
INFO:__main__:Negative sampling time: 0.05585002899169922s
INFO:__main__:Epoch 66 [0.7 s]  loss=220.0372
INFO:__main__:Negative sampling time: 0.05385589599609375s
INFO:__main__:Epoch 67 [0.7 s]  loss=217.7098
INFO:__main__:Negative sampling time: 0.054852962493896484s
INFO:__main__:Epoch 68 [0.7 s]  loss=217.3905
INFO:__main__:Negative sampling time: 0.054853200912475586s
INFO:__main__:Epoch 69 [0.7 s]  loss=213.0157
INFO:__main__:Negative sampling time: 0.05385565757751465s
INFO:__main__:Epoch 70 [0.7 s]  loss=212.6885
INFO:__main__:Negative sampling time: 0.05380535125732422s
INFO:__main__:Epoch 71 [0.7 s]  loss=211.7400
INFO:__main__:Negative sampling time: 0.05485391616821289s
INFO:__main__:Epoch 72 [0.7 s]  loss=208.0942
INFO:__main__:Negative sampling time: 0.052828311920166016s
INFO:__main__:Epoch 73 [0.7 s]  loss=204.2689
INFO:_

INFO:__main__:Negative sampling time: 0.05385470390319824s
INFO:__main__:Epoch 130 [0.6 s]  loss=150.8436
INFO:__main__:Negative sampling time: 0.054853200912475586s
INFO:__main__:Epoch 131 [0.6 s]  loss=149.8767
INFO:__main__:Negative sampling time: 0.055947303771972656s
INFO:__main__:Epoch 132 [0.6 s]  loss=150.5773
INFO:__main__:Negative sampling time: 0.05585169792175293s
INFO:__main__:Epoch 133 [0.7 s]  loss=150.4635
INFO:__main__:Negative sampling time: 0.05385899543762207s
INFO:__main__:Epoch 134 [0.7 s]  loss=150.8943
INFO:__main__:Negative sampling time: 0.05385541915893555s
INFO:__main__:Epoch 135 [0.6 s]  loss=150.5533
INFO:__main__:Negative sampling time: 0.055852651596069336s
INFO:__main__:Epoch 136 [0.6 s]  loss=149.6476
INFO:__main__:Negative sampling time: 0.05585050582885742s
INFO:__main__:Epoch 137 [0.7 s]  loss=149.4741
INFO:__main__:Negative sampling time: 0.05485367774963379s
INFO:__main__:Epoch 138 [0.7 s]  loss=149.5698
INFO:__main__:Negative sampling time: 0.055

INFO:__main__:Negative sampling time: 0.05585050582885742s
INFO:__main__:Epoch 195 [0.6 s]  loss=137.2461
INFO:__main__:Negative sampling time: 0.054853200912475586s
INFO:__main__:Epoch 196 [0.6 s]  loss=138.1275
INFO:__main__:Negative sampling time: 0.054853200912475586s
INFO:__main__:Epoch 197 [0.6 s]  loss=136.9936
INFO:__main__:Negative sampling time: 0.05585074424743652s
INFO:__main__:Epoch 198 [0.6 s]  loss=139.0803
INFO:__main__:Negative sampling time: 0.055850982666015625s
INFO:__main__:Epoch 199 [0.7 s]  loss=137.6519
INFO:__main__:Negative sampling time: 0.05485343933105469s
INFO:__main__:Epoch 200 [0.7 s]  loss=137.3520
INFO:__main__:0.08661126500461737, 0.06334256694367531, 0.04998461064943094, 0.044090489381348054
INFO:__main__:0.02958418704771804, 0.0429128435879658, 0.05012370474661961, 0.05849005233594633
INFO:__main__:0.05326869806094183, 0.03122290375060459, 0.022180694434608572, 0.018389591388617798
INFO:__main__:0.0970733902198779, 0.07721010959313482, 0.06541386684

In [ ]:
r = np.zeros([3,3])

In [ ]:
b = np.array([[1,2,3],[1,3],[1,4,2]])

In [ ]:
np.append(r[0],b[0])

In [ ]:
b[0]